# Join the Register of Overseas Entities companies to HMLR titles registered to overseas companies

Notebook to join the new [Register of Overseas Entities data](XXXX) to HM Land Registry titles registered to overseas companies. 

This isn't straightforward because (inexplicably) CoHouse haven't asked the ROE entries to specify their registered titles, and HMLR don't yet have the ROE identifier in their data, so we need to join fuzzily based on company name.  

Nevertheless we can match a reasonable number of titles - though anything matched should be double-checked. 

Last run at the start of January 2023. 

**How to use**

You'll need the following datasets in BigQuery (or contact me if you want to take a copy of ours):

- HM Land Registry [Overseas companies that own property in England and Wales](https://use-land-property-data.service.gov.uk/datasets/ocod).
- Companies House [Basic Companies Data](http://download.companieshouse.gov.uk/en_output.html)
- Companies House [Persons of Significant Control](http://download.companieshouse.gov.uk/en_pscdata.html) (PSC)

Update the constants below with your own BigQuery project ID and table names.

**TODO**

- Clarify: HMLR records legal title and ROE is about beneficial title, so it's possible the two won't be exactly the same?

In [68]:
import os
import pandas as pd
import pandas_gbq

In [69]:
PROJECT_ID = os.getenv('BQ_PROJECT_ID')
OCOD_TABLE = "ocod.ocod_2023_01"
CH_BASIC_TABLE = "co_house.basic_2023_01"
CH_PSCS_TABLE = "co_house.pscs_2023_01"

## Basic numbers

First let's look at the HMLR overseas companies data.

There are 92,772 titles registered at HMLR as belonging to overseas companies. Of these, 90,725 were added after January 1999 and therefore would normally be expected to be registered on ROE. These titles belong to 30,413 unique companies, so that's our baseline for how many companies we will expect to see on the ROE register by the end of January.

For comparison, on past runs:

- on Nov 21 2022, there were 2609 companies registered.

In [ ]:
sql = "SELECT * FROM `%s.%s`" % (PROJECT_ID, OCOD_TABLE)
df_titles = pandas_gbq.read_gbq(sql, project_id=PROJECT_ID, progress_bar_type=None)
len(df_titles)

In [ ]:
df_titles.date_proprietor_added = pd.to_datetime(df_titles.date_proprietor_added)
df_relevant_titles = df_titles.query("date_proprietor_added >= '1999-01-01'")
print(len(df_relevant_titles))
print(df_relevant_titles.proprietor_name_1.nunique())

Now let's look at the Companies House data, which tells us about the companies now on the ROE register, and their beneficial owners (persons of significant control, or PSCs).

At the start of January, there were 9,994 companies with an `OE*` identifier in the Companies House Basic Company Data.

In [ ]:
sql = "SELECT * FROM `%s.%s` where companynumber LIKE 'OE%%'" % (PROJECT_ID, CH_BASIC_TABLE)
df_companies = pandas_gbq.read_gbq(sql, project_id=PROJECT_ID, progress_bar_type=None)
len(df_companies)

And there are currently 32,570 PSCs in the PSC data with an `OE*` company number. Of these all but about 400 can be joined to companies in the Basic data - so the CoHouse basic and PSC ROE entries are roughly in sync. 

For comparison, on past runs:

- on Nov 21 2022, there were 9593 PSCs registered.

In [ ]:
sql = """
SELECT *
FROM `%s.co_house.pscs_2023_01` 
WHERE  company_number LIKE 'OE%%'
ORDER BY company_number DESC
""" % PROJECT_ID
df_pscs = pandas_gbq.read_gbq(sql, project_id=PROJECT_ID, progress_bar_type=None)
len(df_pscs)

In [ ]:
sql = """
SELECT
  pscs.*, companies.*
FROM
  `%s.%s` AS pscs 
INNER JOIN
  `%s.%s` AS companies
ON
  companies.company_number=pscs.companynumber
WHERE
  pscs.companynumber LIKE 'OE%%'
""" % (PROJECT_ID, CH_BASIC_TABLE, PROJECT_ID, CH_PSCS_TABLE)
df_pscs_with_companies = pandas_gbq.read_gbq(sql, project_id=PROJECT_ID, progress_bar_type=None)
len(df_pscs_with_companies)

## Link the two datasets, and look at the coverage

Now we attempt to see how well the CoHouse and HMLR datasets can be matched up. 

First: how many of our 90k OCOD titles can be linked to a ROE company on direct name match?

The answer is 26,069, so just under 30%.

Second: what happens if we try to match on company name _and_ country name? This produces 20,000 results. Eyeballing the difference suggests that almost all of the delta actually match on country anyway, just the country-names are messy and incompatible between the two datasets. 

So there are probably some false matches in this 26,069, but not loads.

In [ ]:
sql = """
SELECT
  companies.*,  ocod.*
FROM
  `%s.%s` AS companies
INNER JOIN
  `%s.%s` AS ocod
ON
  ocod.proprietor_name_1=companies.companyname
  -- AND ocod.country_incorporated_1=companies.countryoforigin
WHERE
  companies.companynumber LIKE 'OE%%'
""" % (PROJECT_ID, CH_BASIC_TABLE, PROJECT_ID, OCOD_TABLE)
df_titles_joined = pandas_gbq.read_gbq(sql, project_id=PROJECT_ID, progress_bar_type=None)
len(df_titles_joined)

Eyeball the matched results. 

In [ ]:
cols = ['companyname', 'companynumber', 'regaddresscounty', 'regaddresscountry', 
        'title_number', 'property_address', 'country_incorporated_1']
df_titles_joined.head()[cols]

Experiment with ways to make country matching better.

In [ ]:
# Try tidying  up some of the country names so they match across the two datasets.
df_titles_joined.regaddresscountry = \
    df_titles_joined.regaddresscountry.str.replace("VIRGIN ISLANDS, BRITISH", "BRITISH VIRGIN ISLANDS")
df_titles_joined.regaddresscountry = \
    df_titles_joined.regaddresscountry.str.replace("SAINT ", "ST ")

In [ ]:
# Try matching by country on regaddresscounty as well as country, as the fields seem to be used interchangeably.
# That improvees things a bit.
df_unmatched = df_titles_joined\
    .query("regaddresscountry != country_incorporated_1 & regaddresscounty != country_incorporated_1")
len(df_unmatched)

In [ ]:
df_unmatched[cols]

## How many of our ROE entries have we been able to find titles for?

In these matched titles, we have 6,422 unique company names - so that's about two-thirds of all the ROE companies in the basic data.

That suggests this matching isn't that great, because we should be able to find an OCOD entry for every ROE company. 

More work needed!

In [61]:
df_titles_joined.companynumber.nunique()

6422

## Join ROE companies with PSC information and related OCOD titles

Get a list of titles per PSC with any related company and OCOD information that we can match, and save to a file.

In [62]:
sql = """
SELECT
  companies.*,  ocod.*, pscs.*
FROM
  `%s.%s` AS companies
INNER JOIN
  `%s.%s` AS pscs
ON
  companies.companynumber=pscs.company_number
LEFT JOIN
  `%s.%s` AS ocod
ON
  ocod.country_incorporated_1=companies.countryoforigin
  AND ocod.proprietor_name_1=companies.companyname
WHERE
  companies.companynumber LIKE 'OE%%'
""" % (PROJECT_ID, CH_BASIC_TABLE, PROJECT_ID, CH_PSCS_TABLE, PROJECT_ID, OCOD_TABLE)

In [63]:
df_pscs_with_companies = pandas_gbq.read_gbq(sql, project_id=PROJECT_ID, progress_bar_type=None)
len(df_pscs_with_companies)

85754

In [64]:
df_pscs_with_companies.to_csv("./data/roe_pscs_with_companies_and_any_matched_titles.csv", index=False)